# Parse LLM responses and save as jsons

In [1]:
import os
import json
import re
from keys import (
    DEPLOYMENT_GPT_4o_MINI,
    DEPLOYMENT_GPT_4o
)

### Helper Functions

In [2]:
def get_processed_papers(deployment):
    processed_path = os.path.join("LLM_Responses", deployment)
    processed_papers = os.listdir(processed_path)
    if ".DS_Store" in processed_papers:
        processed_papers.remove(".DS_Store")
    return processed_papers

def load_result(deployment, paper_txt):
    txt_path = os.path.join("LLM_Responses", deployment, paper_txt)
    with open(txt_path, "r", encoding="utf-8") as f:
        data = f.read()
    return data

def save_parsed_result(parsed_file, deployment, parsed_respnse):
    file_to_save = os.path.join("LLM_Responses_Parsed", deployment, parsed_file)
    with open(file_to_save, "w", encoding="utf-8") as f:
        json.dump(parsed_respnse, f, indent=2, ensure_ascii=False)

### Parsing Functions

In [3]:
def extract_rating(text):
    # Step 1: Normalize the text
    text = text.replace('\n', ' ')                # Remove line breaks
    text = re.sub(r'\*+', '', text)              # Remove all bold marks like **, ****

    # Step 2: Try matching known rating patterns
    patterns = [
        r"(?i)\bRating\b\s*[:\-]?\s*(\d+(?:\.\d+)?)",                           # Rating: 8 or 7.5
        r"(?i)Rating\s*[:\-]?\s*(\d+(?:\.\d+)?)\s*/\s*10",                      # Rating: 8/10 or 7.5/10
        r"(?i)\d+\.\s*Rating\s*[:\-]?\s*(\d+(?:\.\d+)?)\s*/\s*10",              # 1. Rating: 8/10 or 7.5/10
        r"(?i)\bRating\b.*?(\d+(?:\.\d+)?)\s*/?\s*10?",                         # fallback Rating ... 8/10 or 7.5/10
        r"(?i)\bScore\b\s*[:\-]?\s*(\d+(?:\.\d+)?)",                           # Rating: 8 or 7.5
        r"(?i)\bScore\b\s*[:\-]?\s*(\d+(?:\.\d+)?)\s*/\s*10",                   # Score: 7/10 or 7.5/10
        r"(?i)\bScore\b.*?(\d+(?:\.\d+)?)\s*/?\s*10?",                          # fallback Score ... 7/10 or 7.5/10
        r"(?i)\b(\d+(?:\.\d+)?)\s*/\s*10\b"                                     # standalone 8/10 or 7.5/10
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            try:
                return float(match.group(1))
            except ValueError:
                print(text)
                continue
    return None


def extract_decision(text):
    # Step 1: Normalize the text
    text = text.replace('\n', ' ')                 # Remove line breaks
    text = re.sub(r'\*+', '', text)                # Remove bold marks like **, ****
    text = text.lower().strip()                    # Convert to lowercase

    # Step 2: If "Meta Review" is found, extract text after it
    meta_match = re.search(r'\bmeta review\b', text)
    if meta_match:
        text = text[meta_match.end():].strip()  # Only keep text after "Meta Review"

        # Step 3: Known decision-related patterns
        patterns = [
            r"\brecommendation\b\s*[:\-]?\s*(borderline accept)",
            r"\brecommendation\b\s*[:\-]?\s*(accept with minor revisions)",
            r"\brecommendation\b\s*[:\-]?\s*(accept with major revisions)",
            r"\brecommendation\b\s*[:\-]?\s*(accept)",
            r"\brecommendation\b\s*[:\-]?\s*(reject)",
            r"(\bshould be accepted\b)",
            r"(\brecommended for acceptance\b)",
            r"(\brecommend accepting\b)",
            r"(\bacceptance\b)",
            r"(\baccepted\b)",
            r"(\baccept\b)",
            r"(\bacceptable\b)",
            r"(\bsuggest accepting the paper\b)",
            r"(\bminor revisions\b)",
            r"(\bminor revision\b)",
            r"(\bmajor revisions\b)",
            r"(\bmajor revision\b)",
            r"(\brecommend reject\b)",
            r"(\breject\b)",
            r"(\brevise and resubmit\b)",
            r"(\bsuggest rejection\b)"
        ]

        # Step 4: Match patterns in the post-Meta Review text
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                decision_text = match.group(1).strip() if match.lastindex else match.group(0).strip()
                if any(x in decision_text for x in ["accept", "minor revision", "suggest accepting the paper"]):
                    return "accept"
                elif any(x in decision_text for x in ["reject", "major revision", "revise and resubmit", "suggest rejection"]):
                    return "reject"
        
    else:
        print("[-] No Meta Review")
    return None


def parse_paper(paper_text):
    try:
        # Step 1: Try to split using "Review" headers
        review_sections = re.split(r"(?i)(?:#+\s*)?Review\s+#?\d+\b", paper_text)

        # Step 2: If that didn't yield multiple reviews, try "Reviewer" headers instead
        if len(review_sections) <= 1:
            review_sections = re.split(r"(?i)(?:#+\s*)?Reviewer\s+#?\d+\b", paper_text)

        # Step 3: If that didn't yield multiple reviews, try "Review Opinion" headers instead
        if len(review_sections) <= 1:
            review_sections = re.split(r"(?i)(?:#+\s*)?Review Opinion\s+#?\d+\b", paper_text)

        if len(review_sections) <= 1:
            print("No review")
            return None  # No review found
        

        reviews = review_sections[1:]  # Skip the first part before the first review

        if len(reviews) < 4:
            print("< 4 reviews")
            return None  # Less than 4 reviews

        parsed_reviews = []
        accept_count, reject_count = 0, 0
        for i, review in enumerate(reviews[:4]):  # Only take first 4
            rating = extract_rating(review)
            if rating is None:
                print("No rating")
                return None
            
            decision = extract_decision(review)
            if decision is None:
                print(f"No decision ({i+1})")
                return None
            
            if decision == 'accept':
                accept_count += 1
            if decision == 'reject':
                reject_count += 1

            parsed_reviews.append({
                "text": review.strip(),
                "rating": rating,
                "decision": decision
            })
        
        if accept_count >= reject_count:
            final_decision = "accept"
        else:
            final_decision = "reject"

        parsed_paper = {
            "reviews": parsed_reviews,
            "decision": final_decision
        }


        return parsed_paper

    except Exception as e:
        print(e)
        return None  # On any parsing error


# Run code

In [ ]:
DEPLOYMENT = DEPLOYMENT_GPT_4o_MINI

In [5]:
processed_papers = get_processed_papers(DEPLOYMENT)
for i, paper in enumerate(processed_papers):
    paper_text = load_result(DEPLOYMENT, paper)
    paper_parsed = parse_paper(paper_text)
    if paper_parsed:
        json_file_name = f"{paper.split('.')[0]}.json"
        save_parsed_result(json_file_name, DEPLOYMENT, paper_parsed)
    else:
        print(f"[-] Error parsing: {paper}")